In [1]:
import os, json

In [2]:
# 读取所有的原始样本
all_samples, no_entities_samples = {}, {}
temporal_files, skewed_files = os.listdir('./temporal_splits/'), os.listdir('./skewed_splits/')
for file in temporal_files:
    with open('./temporal_splits/' + file, encoding='utf-8') as f:
        for sample in json.load(f):
            tags = sample['tag_sequence'].split(' ')
            if tags.count('O') < len(tags):
                all_samples[sample['sentence']] = sample['tag_sequence'].replace('B-', '').replace('I-','')
            else:
                no_entities_samples[sample['sentence']] = sample['tag_sequence']
for file in skewed_files:
    with open('./skewed_splits/' + file, encoding='utf-8') as f:
        for sample in json.load(f):
            tags = sample['tag_sequence'].split(' ')
            if tags.count('O') < len(tags):
                all_samples[sample['sentence']] = sample['tag_sequence'].replace('B-', '').replace('I-','')
            else:
                no_entities_samples[sample['sentence']] = sample['tag_sequence']
print(len(all_samples))
print(len(no_entities_samples))

9446
5207


In [3]:
tag_set = set()
for sentence in all_samples:
    tags = all_samples[sentence].split()
    for tag in tags:
        if tag != 'O':
            tag_set.add(tag)
sample_dict = {}
for tag in tag_set:
    samples = []
    for sentence in all_samples:
        tags = all_samples[sentence].split()
        if tag in tags:
            tokens = sentence.split()
            token_tag = []
            for i in range(len(tags)):
                token_tag.append(tokens[i] + '\t' +tags[i])
            samples.append('\n'.join(token_tag))
    sample_dict[tag] = samples
count = 0
for tag in sample_dict:
    count += len(sample_dict[tag])
print(count)
count = 0


tag_list = list(tag_set)
for i in range(len(tag_list)):
    ti_samples = set(sample_dict[tag_list[i]])
    for j in range(i + 1, len(tag_list)):
        tj_samples = set(sample_dict[tag_list[j]])
        inter = ti_samples.intersection(tj_samples)
        c = len(inter)
        if c > 0:
            a, b = len(ti_samples), len(tj_samples)
            for sample in inter:
                if a > b:
                    ti_samples.remove(sample)
                    a -= 1
                else:
                    tj_samples.remove(sample)
                    b -= 1
            sample_dict[tag_list[i]] = list(ti_samples)
            sample_dict[tag_list[j]] = list(tj_samples)
for tag in sample_dict:
    count += len(sample_dict[tag])
print(count)

14707
9446


In [4]:
few_sample_tags = set()
tag_count = {}
for tag in tag_set:
    tag_count[tag] = len(sample_dict[tag])
    if tag_count[tag] < 200:
        few_sample_tags.add(tag)
print(few_sample_tags)
print(tag_count)

{'Function_Name', 'User_Name', 'Organization', 'Website', 'Error_Name', 'Licence', 'Output_Block', 'Algorithm', 'Keyboard_IP'}
{'Version': 260, 'Data_Type': 429, 'Data_Structure': 621, 'Function_Name': 188, 'Library': 641, 'Code_Block': 613, 'Application': 744, 'User_Interface_Element': 667, 'Organization': 45, 'File_Type': 416, 'HTML_XML_Tag': 232, 'Error_Name': 104, 'Device': 277, 'Class_Name': 283, 'File_Name': 425, 'User_Name': 131, 'Variable_Name': 498, 'Algorithm': 60, 'Library_Function': 498, 'Library_Variable': 312, 'Language': 472, 'Value': 457, 'Licence': 1, 'Operating_System': 251, 'Output_Block': 84, 'Website': 119, 'Library_Class': 574, 'Keyboard_IP': 44}


In [5]:
for tag in few_sample_tags:
    tag_set.remove(tag)
    tag_count.pop(tag)
    sample_dict.pop(tag)
for few_sample_tag in few_sample_tags:
    for tag in sample_dict:
        clear_sentences = []
        for sentence in sample_dict[tag]:
            wts = sentence.split('\n')
            for i in range(len(wts)):
                w_t = wts[i].split('\t')
                w_t[1] = w_t[1].replace(few_sample_tag, 'O')
                wts[i] = '\t'.join(w_t)
            clear_sentences.append('\n'.join(wts))
        sample_dict[tag] = clear_sentences
count = 0
for tag in tag_count:
    count += tag_count[tag]
print(count)

8670


In [6]:
for entity_type in sample_dict:
    print(entity_type,':',len(sample_dict[entity_type]))
distribution = {}
for entity_type in sample_dict:
    samples = sample_dict[entity_type]
    type_dict = {}
    for sample in samples:
        token_tags = sample.split('\n')
        tag_set = set()
        for token_tag in token_tags:
            tag = token_tag.split('\t')[1]
            if tag != 'O':
                tag_set.add(tag)
        for tag in tag_set:
            type_dict[tag] = type_dict.get(tag, 0) + 1
    distribution[entity_type] = type_dict
with open('./continual/distribution.json', encoding='utf-8', mode='w') as f:
    json.dump(distribution, f)

Version : 260
Data_Type : 429
Data_Structure : 621
Library : 641
Code_Block : 613
Application : 744
User_Interface_Element : 667
File_Type : 416
HTML_XML_Tag : 232
Device : 277
Class_Name : 283
File_Name : 425
Variable_Name : 498
Library_Function : 498
Library_Variable : 312
Language : 472
Value : 457
Operating_System : 251
Library_Class : 574


In [7]:
import random
no_dir = './continual/disjoint/'
o_dir = './continual/joint/'
for entity_type in sample_dict:
    no_path = os.path.join(no_dir, entity_type)
    o_path = os.path.join(o_dir, entity_type)
    if not os.path.exists(no_path):
        os.makedirs(no_path)
    if not os.path.exists(o_path):
        os.makedirs(o_path)
    samples = sample_dict[entity_type]
    train_samples, valid_samples = set(), set()
    while len(train_samples) < 0.7 * len(samples):
        sample = random.choice(samples)
        if sample not in train_samples:
            train_samples.add(sample)
    while len(valid_samples) < 0.1 * len(samples):
        sample = random.choice(samples)
        if sample not in train_samples and sample not in valid_samples:
            valid_samples.add(sample)
    test_samples = set(samples).difference(train_samples.union(valid_samples))
    with open(os.path.join(o_path, 'train.txt'), encoding='utf-8', mode='w') as f:
        f.writelines('\n\n'.join(train_samples))
    with open(os.path.join(no_path, 'train.txt'), encoding='utf-8', mode='w') as f:
        samples = list(train_samples)
        for i in range(len(samples)):
            token_tags = samples[i].split('\n')
            for j in range(len(token_tags)):
                tag = token_tags[j].split('\t')[1]
                if tag != 'O' and tag != entity_type:
                    token_tags[j] = token_tags[j].replace(tag, 'O')
            samples[i] = '\n'.join(token_tags)
        f.writelines('\n\n'.join(samples))
    with open(os.path.join(o_path, 'dev.txt'), encoding='utf-8', mode='w') as f:
        f.writelines('\n\n'.join(valid_samples))
    with open(os.path.join(no_path, 'dev.txt'), encoding='utf-8', mode='w') as f:
        samples = list(valid_samples)
        for i in range(len(samples)):
            token_tags = samples[i].split('\n')
            for j in range(len(token_tags)):
                tag = token_tags[j].split('\t')[1]
                if tag != 'O' and tag != entity_type:
                    token_tags[j] = token_tags[j].replace(tag, 'O')
            samples[i] = '\n'.join(token_tags)
        f.writelines('\n\n'.join(samples))
    with open(os.path.join(o_path, 'test.txt'), encoding='utf-8', mode='w') as f:
        f.writelines('\n\n'.join(test_samples))
    with open(os.path.join(no_path, 'test.txt'), encoding='utf-8', mode='w') as f:
        samples = list(test_samples)
        for i in range(len(samples)):
            token_tags = samples[i].split('\n')
            for j in range(len(token_tags)):
                tag = token_tags[j].split('\t')[1]
                if tag != 'O' and tag != entity_type:
                    token_tags[j] = token_tags[j].replace(tag, 'O')
            samples[i] = '\n'.join(token_tags)
        f.writelines('\n\n'.join(samples))

In [9]:
no_entities_samples_list = []
for sentence in no_entities_samples:
    tags = no_entities_samples[sentence].split()
    tokens = sentence.split()
    token_tag = []
    for i in range(len(tags)):
        token_tag.append(tokens[i] + '\t' +tags[i])
    no_entities_samples_list.append('\n'.join(token_tag))
with open('no_entities.txt', encoding='utf-8', mode='w') as f:
    f.writelines('\n\n'.join(no_entities_samples_list))